In [1]:
import numpy as np
import pandas as pd
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
# direct the path to appropriate data location
interact_oct2019 = pd.read_csv('./data/interactions-2019-Oct.csv')
interact_nov2019 = pd.read_csv('./data/interactions-2019-Nov.csv')
interact_dec2019 = pd.read_csv('./data/interactions-2019-Dec.csv')
interact_jan2020 = pd.read_csv('./data/interactions-2020-Jan.csv')
interact_feb2020 = pd.read_csv('./data/interactions-2020-Feb.csv')
items = pd.read_csv('./data/items_catalog.csv')
interact_oct2019.head()

,interaction_time,user_id,item_id,interaction_type,price
0,2019-10-01 00:00:00,0fdce5b4-6c26-46fc-99d9,5773203,AddToCart,2.62
1,2019-10-01 00:00:03,0fdce5b4-6c26-46fc-99d9,5773353,AddToCart,2.62
2,2019-10-01 00:00:07,e01d554a-9df4-427d-b13d,5881589,AddToCart,13.48
3,2019-10-01 00:00:07,0fdce5b4-6c26-46fc-99d9,5723490,AddToCart,2.62
4,2019-10-01 00:00:15,e01d554a-9df4-427d-b13d,5881449,AddToCart,0.56


In [3]:
interactions = pd.concat([interact_oct2019, interact_nov2019, interact_dec2019, 
                          interact_jan2020, interact_feb2020]).reset_index()

del interact_oct2019
del interact_nov2019
del interact_dec2019
del interact_jan2020
del interact_feb2020
gc.collect() # garbage collect

49

In [4]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   index             int64  
 1   interaction_time  object 
 2   user_id           object 
 3   item_id           int64  
 4   interaction_type  object 
 5   price             float64
dtypes: float64(1), int64(2), object(3)
memory usage: 947.2+ MB


In [5]:
# changing variables to appropriate types, this can save memory
interactions['interaction_time'] = pd.to_datetime(interactions['interaction_time'])
interactions["interaction_type"] = interactions["interaction_type"].astype("category")
interactions["price"] = interactions["price"].astype("float32")

In [6]:
interactions.info() # interactions df memory reduced from 947 MB to 730 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   index             int64         
 1   interaction_time  datetime64[ns]
 2   user_id           object        
 3   item_id           int64         
 4   interaction_type  category      
 5   price             float32       
dtypes: category(1), datetime64[ns](1), float32(1), int64(2), object(1)
memory usage: 730.2+ MB


### First visit, last visit, average gap between visits (null for users with only 1 visit)

In [7]:
# first visit datetime, last visit datetime
# for each user, difference between multiple interactions is tracked and then the mean of this is computed
user_df = interactions.groupby('user_id').interaction_time.agg(first_visit_date=np.min, 
                                                               last_visit_date=np.max,
                                                               count='count',
                                                               avg_gap=lambda user: user.diff().mean()).reset_index()
user_df.head(3)

,user_id,first_visit_date,last_visit_date,count,avg_gap
0,00000a88-2beb-4f3a-bf8a,2019-12-16 09:15:42,2020-02-26 12:59:50,25,3 days 00:09:20.333333
1,00000cfa-9a75-4ee4-8ff0,2020-02-07 15:53:22,2020-02-29 04:19:54,3,10 days 18:13:16
2,00001957-83e2-4e36-aacf,2020-01-31 07:07:19,2020-02-11 15:06:17,25,0 days 11:19:57.416666


### Average Monthly Spend - Only considers months after user makes first visit
Average monthly spend = Total Spend/Months till latest date (Feb) after 1st visit

In [8]:
# months_after_first_visit - time in months between the first visit and the end of the latest month of data
latest_date = interactions.interaction_time.max()
user_df['months_after_first_visit'] = round((latest_date - user_df.first_visit_date)/np.timedelta64(1, 'M'),1)
user_df.head(2)

,user_id,first_visit_date,last_visit_date,count,avg_gap,months_after_first_visit
0,00000a88-2beb-4f3a-bf8a,2019-12-16 09:15:42,2020-02-26 12:59:50,25,3 days 00:09:20.333333,2.5
1,00000cfa-9a75-4ee4-8ff0,2020-02-07 15:53:22,2020-02-29 04:19:54,3,10 days 18:13:16,0.7


In [9]:
# filtering purchase interactions and capturing total spend per user, later used to capture monthly spend
user_total_spend = interactions[interactions.interaction_type == 'Purchase'] \
                    .groupby('user_id').price.agg(total_spend = np.sum).reset_index()

user_total_spend.head(2)

,user_id,total_spend
0,000148b5-6ec2-47a5-83ba,64.260002
1,0002c237-9155-4d19-a0b4,133.540009


In [10]:
user_df = user_df.merge(user_total_spend, on = 'user_id', how = 'left')
user_df['total_spend'] = user_df.total_spend.fillna(0) # users with no spending

In [11]:
user_df['avg_monthly_spending'] = user_df.total_spend/user_df.months_after_first_visit

### Favorite Brand

In [12]:
# for items that have multiple brands, the first occuring brand that doesn't have the value "(not set)" is taken
items['item_id_count'] = items.item_id.map(items.item_id.value_counts())

items = items[~((items['item_id_count'] > 1) & (items['brand'] == '(not set)'))] \
         .drop_duplicates(subset='item_id', keep="first")[['item_id', 'brand']]

In [13]:
interactions = interactions.merge(items, on = 'item_id', how = 'left')

In [14]:
interactions.head(2)

,index,interaction_time,user_id,item_id,interaction_type,price,brand
0,0,2019-10-01 00:00:00,0fdce5b4-6c26-46fc-99d9,5773203,AddToCart,2.62,Mykek
1,1,2019-10-01 00:00:03,0fdce5b4-6c26-46fc-99d9,5773353,AddToCart,2.62,Mykek


The favorite brand for each user is determined in a hierarchical fashion. The following is the priority in which the brands are selected for each user:

1) Most frequently occuring brand with respect to purchase interactions \
2) Most frequently occuring brand with respect to "Add to Cart" interactions \
3) Most frequently occuring brand with respect to other types of interactions

In [15]:
purchase_interactions = interactions[interactions['interaction_type'].str.contains("Purchase")]

# frequency of each brand that was purchased from and selecting the one with the highest count as the favorite brand
purchase_brands = purchase_interactions.groupby('user_id').brand.value_counts().reset_index(name="count")
purchase_fav_brand = purchase_brands.drop_duplicates('user_id')[['user_id','brand']]

In [16]:
addcart_interactions = interactions[interactions['interaction_type'].str.contains("AddToCart")]

# removing users that are captured through purchase interactions
addcart_interactions = addcart_interactions[~addcart_interactions['user_id'].isin(purchase_fav_brand['user_id'])]

# frequency of each brand that was added to cart and selecting the one with the highest count as the favorite brand
addcart_brands = addcart_interactions.groupby('user_id').brand.value_counts().reset_index(name="count")
addcart_fav_brand = addcart_brands.drop_duplicates('user_id')[['user_id','brand']]

In [17]:
# filtering for other interactions
other_interactions = interactions[~interactions['interaction_type'].str.contains("Purchase|AddToCart")]

# removing users that are captured through purchase and add to cart interactions
other_interactions = other_interactions[~other_interactions['user_id'].isin(purchase_fav_brand['user_id'])]
other_interactions = other_interactions[~other_interactions['user_id'].isin(addcart_fav_brand['user_id'])]

# frequency of each brand through other interactions and selecting the one with the highest count as the favorite brand
otherint_brands = other_interactions.groupby('user_id').brand.value_counts().reset_index(name="count")
otherint_fav_brand = otherint_brands.drop_duplicates('user_id')[['user_id','brand']]


In [18]:
fav_brand_df = pd.concat([purchase_fav_brand, addcart_fav_brand, otherint_fav_brand])

In [19]:
user_df = user_df.merge(fav_brand_df, on = 'user_id', how = 'left')
user_df = user_df[['user_id', 'first_visit_date', 'last_visit_date', 'avg_gap', 'avg_monthly_spending', 'brand']]

In [23]:
user_df.head()

,user_id,first_visit_date,last_visit_date,avg_gap,avg_monthly_spending,brand
0,00000a88-2beb-4f3a-bf8a,2019-12-16 09:15:42,2020-02-26 12:59:50,3 days 00:09:20.333333,0.0,(not set)
1,00000cfa-9a75-4ee4-8ff0,2020-02-07 15:53:22,2020-02-29 04:19:54,10 days 18:13:16,0.0,Beb ip aq
2,00001957-83e2-4e36-aacf,2020-01-31 07:07:19,2020-02-11 15:06:17,0 days 11:19:57.416666,0.0,(not set)
3,00001af7-43f5-4c04-be0e,2020-02-13 18:08:22,2020-02-20 16:16:13,0 days 07:13:23.086956,0.0,Kenu hijy
4,0000200c-8a05-44d7-bbd6,2019-11-29 12:45:04,2019-11-29 12:45:04,NaT,0.0,Kyyo
